In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive_output, FloatSlider, Dropdown, HTML, Layout, Output, HBox, VBox
from IPython.display import display
from sklearn.linear_model import LinearRegression

In [2]:
# Load data
full_data = pd.read_csv("Constant Strain Rate Data.csv")
epsT_full = np.array(full_data["Strain"], dtype=float)
SR = 1.0  # set strain rate (you can add a slider for this if desired)
temps = [c for c in full_data.columns if c != "Strain"]
low_temps = temps[:3]
high_temps = temps[3:]
full_temps = [low_temps, high_temps]
param_names = ["E", "k", "K", "n1", "A", "C", "n2", "B"]
params_by_temp = {
    "200": dict(E=30000.0, k=50.0, K=200.0, n1=1.0, A=5.0, C=0.5, n2=1.5, B=200.0),
    "250": dict(E=28000.0, k=45.0, K=210.0, n1=1.0, A=4.5, C=0.6, n2=1.5, B=190.0),
    "300": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "350": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "400": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "450": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "500": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "535": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0)
}

In [3]:
# numerical scheme 
def simulate_sigma_euler(
    epsT: np.ndarray,          # true strain array (monotonic increasing)
    SR: float,                 # strain rate (1/s)
    E: float, k: float, K: float, n1: float,
    A: float, C: float, n2: float,
    B: float,
    eps_p0: float = 0.0,
    rho0: float = 1e-4,
    rho_floor: float = 1e-12,
    n_sub: int = 1,            # set to e.g. 10, 50 if it jumps
):
    """
    Forward Euler on eps_p and rho, with algebraic R = B*sqrt(rho).
    Returns sigma_model, eps_p_hist, rho_hist, R_hist
    All stress-like quantities must be in consistent units (e.g., MPa everywhere).
    """

    # ensure epsT is np.array
    epsT = np.asarray(epsT, dtype=float)
    # number of sigma values to generate
    N = len(epsT)

    sigma = np.zeros(N)
    eps_p = np.zeros(N)
    rho = np.zeros(N)
    R = np.zeros(N)

    # intialise plastic strain as zero
    eps_p[0] = eps_p0

    # why is rho bounded?
    rho[0] = max(rho0, rho_floor)
    R[0] = B * np.sqrt(rho[0])
    sigma[0] = E * (epsT[0] - eps_p[0])

    # sub-stepping routine for stability 
    for i in range(1, N):
        # compute strain step size
        d_epsT = epsT[i] - epsT[i-1]
        if d_epsT <= 0:
            raise ValueError("epsT must be strictly increasing.")
        # compute time step size
        dt = d_epsT / SR

        # substepping (optional but very effective)
        dt_sub = dt / n_sub
        epsT_sub0 = epsT[i-1]

        epp = eps_p[i-1]
        r = rho[i-1]
        
        # if we have instability we can reduce the time step
        for s in range(n_sub):
            # linearl
            epsT_s = epsT_sub0 + (s + 1) * (d_epsT / n_sub)

            r = max(r, rho_floor)
            R_s = B * np.sqrt(r)
            sigma_s = E * (epsT_s - epp)

            drive = (sigma_s - R_s - k) / K
            epp_dot = max(drive, 0) ** n1

            r_dot = A * (1.0 - r) * epp_dot - C * (r ** n2)

            # Euler update
            epp = epp + epp_dot * dt_sub
            r = r + r_dot * dt_sub

            r = max(r, rho_floor)

        eps_p[i] = epp
        rho[i] = r
        R[i] = B * np.sqrt(rho[i])
        sigma[i] = E * (epsT[i] - eps_p[i])

    return sigma, eps_p, rho, R


In [4]:
# linear regression for fitting
def linear_regression(x, y):
    model = LinearRegression()
    model.fit(x, y)
    intercept = model.intercept_
    grad = model.coef_
    intercept = np.ravel(intercept).item()
    grad = np.ravel(grad).item()
    return float(intercept), float(grad)

In [5]:
# Create an Output widget for the plot
output_model = Output()
output_params = Output()

# Interactive plotting function
def plot_simulation(temp, E, k, K, n1, A, C, n2, B):
    with output_model:
        output_model.clear_output(wait=True)  # Clear previous plot
        sigma_exp = full_data[temp].to_numpy()
        mask = ~np.isnan(sigma_exp)
        epsT = epsT_full[mask]
        sigma_exp = sigma_exp[mask]
        
        sigma_model, _, _, _ = simulate_sigma_euler(
            epsT=epsT,
            SR=SR,
            E=E, k=k, K=K, n1=n1, A=A, C=C, n2=n2, B=B,
            rho0=1e-4,
            n_sub=1
        )
        
        plt.figure(figsize=(5, 3))
        plt.title(f"Simulation and experiment plot for {temp}°C", fontsize=12)
        plt.plot(epsT, sigma_model, label='Model', color='blue')
        plt.scatter(epsT, sigma_exp, label='Experiment', color='red', alpha=0.7, s=15)
        plt.xlabel('Strain', fontsize = 10)
        plt.ylabel('Stress (MPa)', fontsize = 10)
        plt.legend()
        plt.grid(True)
        plt.show()

def plot_regression_high(temp, E, k, K, n1, A, C, n2, B):
    with output_params:
        output_params.clear_output(wait=True)
        
        
        params_by_temp[temp] = {
            "E": E, "k": k, "K": K, "n1": n1,
            "A": A, "C": C, "n2": n2, "B": B
        }

        # param vs temperature plots
        fig, axes = plt.subplots(2, 4, figsize=(26, 12), sharex=True)
        axes = axes.ravel()
        for ax, name in zip(axes, param_names):
            # select the variable of interest
            y = np.array([params_by_temp[temp][name] for temp in high_temps])
            # fitting in logspace against, 1/T (Kelvin)
            log_y = np.log(y).reshape(-1, 1)
            numerical_temps = np.array([int(i) for i in high_temps])
            T_inv = (1 / (numerical_temps + 273.15)).reshape(-1,1)
            intercept, grad = linear_regression(T_inv, log_y)
            fit = lambda x : grad * x + intercept
            ax.plot(T_inv, fit(T_inv), linestyle="--")
            ax.scatter(T_inv, log_y, c='r', marker="x")
            ax.text(0.2, 0.85, 
                    f"log({name}) = {intercept:.3f} + {grad:.3f} (1 / T)", 
                    fontsize = 15,
                    transform=ax.transAxes)
            ax.tick_params(labelbottom=True)
            ax.set_title(f"log({name}) vs 1 / T ")
            ax.set_xlabel("1 / T", fontsize = 10)
            ax.set_ylabel(f"log({name})", fontsize = 10)
            ax.grid(True)

        plt.tight_layout()
        plt.show()

def plot_regression_low(temp, E, k, K, n1, A, C, n2, B):
    with output_params:
        output_params.clear_output(wait=True)
        
        
        params_by_temp[temp] = {
            "E": E, "k": k, "K": K, "n1": n1,
            "A": A, "C": C, "n2": n2, "B": B
        }

        # param vs temperature plots
        fig, axes = plt.subplots(2, 4, figsize=(26, 12), sharex=True)
        axes = axes.ravel()
        for ax, name in zip(axes, param_names):
            # select the variable of interest
            y = np.array([params_by_temp[temp][name] for temp in low_temps])
            # fitting in logspace against, 1/T (Kelvin)
            log_y = np.log(y).reshape(-1, 1)
            numerical_temps = np.array([int(i) for i in low_temps])
            T_inv = (1 / (numerical_temps + 273.15)).reshape(-1,1)
            intercept, grad = linear_regression(T_inv, log_y)
            fit = lambda x : grad * x + intercept
            ax.plot(T_inv, fit(T_inv), linestyle="--")
            ax.scatter(T_inv, log_y, c='r', marker="x")
            ax.text(0.2, 0.85, 
                    f"log({name}) = {intercept:.3f} + {grad:.3f} (1 / T)", 
                    fontsize = 15,
                    transform=ax.transAxes)
            ax.tick_params(labelbottom=True)
            ax.set_title(f"log({name}) vs 1 / T ")
            ax.set_xlabel("1 / T", fontsize = 10)
            ax.set_ylabel(f"log({name})", fontsize = 10)
            ax.grid(True)

        plt.tight_layout()
        plt.show()

def full_plot(temp, E, k, K, n1, A, C, n2, B, low):
    plot_simulation(temp, E, k, K, n1, A, C, n2, B)
    if low:
        plot_regression_low(temp, E, k, K, n1, A, C, n2, B)
    else:
        plot_regression_high(temp, E, k, K, n1, A, C, n2, B)


In [6]:
def widget_display(low):
    if low:
        temp_dropdown = Dropdown(options=low_temps, value=low_temps[0], description='Temp:', layout=Layout(width='200px', height='20px'))
        high_low = Dropdown(options=[True], value=True)
    else:
        temp_dropdown = Dropdown(options=high_temps, value=high_temps[0], description='Temp:', layout=Layout(width='200px', height='20px'))
        high_low = Dropdown(options=[False], value=False)
        
    E_slider = FloatSlider(min=1000, max=80000, step=500, value=30000, description='E (MPa):', layout=Layout(width='500px', height='10px'))
    k_slider = FloatSlider(min=0, max=200, step=1, value=50, description='k (MPa):', layout=Layout(width='500px', height='10px'))
    K_slider = FloatSlider(min=10, max=500, step=1, value=200, description='K (MPa):', layout=Layout(width='500px', height='10px'))
    n1_slider = FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n1:', layout=Layout(width='500px', height='10px'))
    A_slider = FloatSlider(min=0.01, max=5, step=0.05, value=5.0, description='A:', layout=Layout(width='500px', height='10px'))
    C_slider = FloatSlider(min=1, max=200, step=1, value=0.5, description='C:', layout=Layout(width='500px', height='10px'))
    n2_slider = FloatSlider(min=0.1, max=5, step=0.1, value=1.5, description='n2:', layout=Layout(width='500px', height='10px'))
    B_slider = FloatSlider(min=10, max=500, step=1, value=200, description='B (MPa):', layout=Layout(width='500px', height='10px'))

    # format layout, sliders in a vertical box
    controls = VBox([
        temp_dropdown,
        E_slider, k_slider, K_slider, n1_slider,
        A_slider, C_slider, n2_slider, B_slider,
    ], layout=Layout(width="700px"))

    # group outputs
    output_model.layout  = Layout(width="700px", height="500px")
    output_params.layout = Layout(width="1000px", height="500px")

    plots = HBox([output_model, controls])

    display(VBox([output_params, plots]))

    # Use interact to create the interactive plot
    full_display = interactive_output(full_plot,
                {"temp": temp_dropdown,
                "E":E_slider, 
                "k":k_slider, 
                "K":K_slider, 
                "n1":n1_slider, 
                "A":A_slider, 
                "C":C_slider, 
                "n2":n2_slider, 
                "B":B_slider,
                "low":high_low})
    

## Low Temperature

In [7]:
widget_display(low=True)

# High Temperature

In [8]:
widget_display(low=False)